In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import cv2
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
import keras
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline
import keras
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization

In [ ]:
# Mount Google Drive (for COLAB)
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Load the Dataset
from glob import glob
train = glob('/content/gdrive/My Drive/wangdataset/Images/*.jpg', recursive=True)
test = glob('/content/gdrive/My Drive/wangdataset/Images/test/*.jpg', recursive=True)

In [ ]:
import math
from math import floor
x_train = []
x_test = []
y_train = []
y_test = []
WIDTH = 128
HEIGHT = 128
for img in train:
    filename = os.path.basename(img)
    label = floor( int(filename.split('.')[0]) / 100 )
    y_train.append(label)
  
for img in test:
    filename = os.path.basename(img)
    label = floor( int(filename.split('.')[0]) / 100 )
    y_test.append(label)

In [ ]:
for img in train:
    full_size_image = cv2.imread(img)
    x_train.append(cv2.resize(full_size_image, (WIDTH,HEIGHT)))
for img in test:
    full_size_image = cv2.imread(img)
    x_test.append(cv2.resize(full_size_image, (WIDTH,HEIGHT)))

In [ ]:
X_train= np.array(x_train)
X_train= X_train/255.0
X_test= np.array(x_test)
X_test= X_test/255.0

In [ ]:
# One Hot Encoding
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical( y_train, num_classes = 10)
y_testHot = to_categorical( y_test, num_classes = 10)

In [ ]:
# Load the Model
base_model = keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(128,128, 3), classes=10)

In [ ]:
last = base_model.get_layer('mixed10').output
# Add classification layers on top of it
x = BatchNormalization()
x = Flatten()(last)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(10, activation='softmax')(x)

model =Model(base_model.input, pred)


In [ ]:
# Compile and Fit the Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['acc'])
a = X_train
b = y_trainHot
history = model.fit(a, b, epochs=200, batch_size=128, validation_split=0.01)

In [ ]:
# Metrics
test_loss, test_acc = model.evaluate(X_test,y_testHot)
print("Test Loss: ", test_loss*100)
print("Test Accuracy: ", test_acc*100)

# Report

In [ ]:
# Evaluation Metrics
target_names = ['Tribes','Beach', 'Monuments', 'Bus', 'Dinosauras'
           ,'Elephant', 'Flower', 'Hourse', 
            'Mountains', 'Food']
# Prediction           
def reports(X_test,y_test):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)

    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss = score[0]*100
    Test_accuracy = score[1]*100
    kc=cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    #mse=mean_squared_error(y_test, y_pred)
    #mae=mean_absolute_error(y_test, y_pred)
    #precision=precision_score(y_test, y_pred, average='weighted')
    #print(classification_report(y_test, y_pred, target_names=target_names))


    
    return classification, confusion, Test_Loss, Test_accuracy ,kc#,mse,mae

In [ ]:
#model = load_model('/content/gdrive/My Drive/wangdataset/model_hb.h5')
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score
# calculate result, loss, accuray and confusion matrix
classification, confusion, Test_loss, Test_accuracy,kc = reports(X_test,y_testHot)
classification = str(classification)
confusion_str = str(confusion)


In [ ]:
print("confusion matrix: ")
print('{}'.format(confusion_str))
print("KAppa Coeefecient=",kc)
print('Test loss {} (%)'.format(Test_loss))
print('Test accuracy {} (%)'.format(Test_accuracy))
#print("Mean Squared error=",mse)
#print("Mean absolute error=",mae)
print(classification)

In [ ]:
# Plot the Confusion Matrix
import matplotlib.pyplot as plt
%matplotlib inline
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.get_cmap("Blues")):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    Normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    if normalize:
        cm = Normalized
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(Normalized, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.4f' if normalize else 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        thresh = cm[i].max() / 2.
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure(figsize=(15,15))
plot_confusion_matrix(confusion, classes=target_names, normalize=False, 
                      title='Confusion matrix, without normalization')
#plt.savefig("/content/gdrive/My Drive/wangdataset/result/confusion_matrix_without_normalization.svg")
plt.show()
plt.figure(figsize=(15,15))
plot_confusion_matrix(confusion, classes=target_names, normalize=True, 
                      title='Normalized confusion matrix')
#plt.savefig("/content/gdrive/My Drive/wangdataset/result/confusion_matrix_with_normalization.png")
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("/content/gdrive/My Drive/wangdataset/model_accuracy_100.svg")
plt.show()

# summarize history for loss 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("/content/gdrive/My Drive/wangdataset/model_loss_100.svg")
plt.show()